In [1]:
import jieba
import math
import os
import json
import jieba.posseg as pseg

from collections import OrderedDict

p_upper = 3.0
p_upperT = "30"
filedir = 'data_tfidf'
filenum = '4K'
filename = 'Result_' + filenum
print(filenum , '-', p_upper)

4K - 3.0


In [5]:
class TF_IDF():
    def __init__(self):
        self.docs = {}
        self.seg_docs = self.get_seg_docs()
        self.stopword = []
        self.tf = []
        self.df = {}
        self.idf = {}
        self.topK_idf = {}
        self.bow = {}
        self.cal_tfidf()

    def read_file(self, path, type):
        if type == 'json':
            with open(path, 'r', encoding='utf-8') as file:
                data = json.loads(file.read())
        elif type == 'txt':
            with open(path, 'r', encoding='utf-8') as file:
                data = file.read()
        return data

    def get_seg_docs(self):
        _seg_docs = []
        FOLDER_NAME = filedir
        DOCUMENT = 'data_3988_512X.json'
        STOPWORD = 'stopword.txt'
        #FILE_DIR = os.path.join(os.path.split(os.path.realpath(__file__))[0], FOLDER_NAME)

        self.docs = self.read_file('.\\' + FOLDER_NAME +'\\' + DOCUMENT, 'json')
        self.stopword = self.read_file('.\\' + FOLDER_NAME + '\\' + STOPWORD, 'txt')
        for i in range(len(self.docs)):
            content_seg = [w for w in jieba.lcut(self.docs[i]['content']) if len(w) > 1 and w not in self.stopword and w.isalpha()]
            _seg_docs.append(content_seg)
        return _seg_docs
    """
    計算tf,idf結果
    tf:[{word1:3,word2:4,word4:2},{word2:5,word3:7, word4:2},{....},.......]
    df:{word1:6個文檔,word2:3個文檔,word3:5個文檔,word4:4個文檔......}
    idf:{word1:idf(word1),word2:idf(word2),word3:idf(word3)..........}
    """
    def cal_tfidf(self):
        for doc in self.seg_docs:
            bow = {}
            for word in doc:
                if not word in bow:
                    bow[word] = 0
                bow[word] += 1
            self.tf.append(bow)
            for word, _ in bow.items():
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1
        for word, df in self.df.items():
            #只出現過在一篇文檔的詞不要(選擇性)
            if df < 2:
                pass
            else:
                self.idf[word] = math.log10(len(self.seg_docs) / df)

    def tf(self, index, word):
        return self.tf[index][word]

    def idf(self, word):
        return self.idf[word]

    def tf_idf(self, index, word):
        return self.tf[index][word]*self.idf[word]

    def get_topK_idf(self, k, reverse = True):
        self.topK_idf = OrderedDict(sorted(self.idf.items(), key=lambda t: t[1], reverse = reverse)[:k])
        return  self.topK_idf

    def get_docment(self):
        return self.docs

    def get_title(self, index):
        return self.docs[index]['title']

    def get_content(self, index):
        return self.docs[index]['content']

    def set_bag_of_word(self, bow):
        self.bow = bow

    def get_text_vector(self, index):
        return [1*self.tf_idf(index, w) if w in jieba.lcut(self.docs[index]['content']) else 0 for w in self.bow]

    def cosine_similarity(self, v1, v2):
        #compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)
        sum_xx, sum_xy, sum_yy = 0.0, 0.0, 0.0
        for i in range(0, len(v1)):
            x, y = v1[i], v2[i]
            sum_xx += math.pow(x, 2)
            sum_yy += math.pow(y, 2)
            sum_xy += x * y
        try:
            return sum_xy / math.sqrt(sum_xx * sum_yy)
        except ZeroDivisionError:
            return 0

In [6]:
tf_idf = TF_IDF()
topK = tf_idf.get_topK_idf(200000, True)
#保存bag of word
tf_idf.set_bag_of_word(set(topK.keys()))
#得到文章第1篇跟第11篇的向量
vec1 = tf_idf.get_text_vector(0)
vec2 = tf_idf.get_text_vector(10)
#計算文件與文件的cosine similarity
score1 = tf_idf.cosine_similarity(vec1, vec1)
score2 = tf_idf.cosine_similarity(vec1, vec2)
#print(topK, score1, score2)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.763 seconds.
Prefix dict has been built succesfully.


In [7]:
f = open(filedir + "\\Result_TFIDF_" + filenum + ".csv", "a", encoding='UTF-8')
f.write("詞,詞性,P\n")

for k, v in topK.items():
    try:
        _k = str(k)
        words = pseg.cut(_k)
        n = ""
        for _w in words:
            n = _w.flag
        s = _k + "," + n + "," + str(v) + "\n"
        f.write(s)
    except Exception as e:
        print(e)
        continue

f.close()

In [8]:
print(tf_idf.get_content(0))

吳鈞婷明知註冊審定號第00000000號商標圖樣係商標權人美商新時代冠帽公司，向我國經濟部智慧財產局申請註冊，取得商標權之註冊商標，指定使用於運動帽之商品，現仍於專用期間內，非經上開商標權人之授權或同意，不得於同一或類似商品，使用相同或近似之商標，而致相關消費者有混淆誤認之虞。詎吳鈞婷竟基於意圖販賣而陳列仿冒前開商標商品之犯意，於103年5月19日23時6分許，在其高雄市○○區○○00○0號住處，利用電腦設備連結網際網路，以「ayu494s」帳號登入露天拍賣網站，在不特定人均能瀏覽之前開拍賣網頁上，以每件新臺幣250元之價格，刊登販賣仿冒前開運動帽之照片及拍賣訊息，供不特定人瀏覽選購而陳列之，並提供其金融機構帳戶供購買者匯款。嗣經新時代公司委任之台宜顧問有限公司員工瀏覽前述網頁後，認該網頁經營者涉嫌違反商標法，乃出於蒐證之目的，於103年12月18日下標購買，待取得吳鈞婷寄交之仿冒前開商標圖樣之運動帽1頂，因而報警處理，並提出上開運動帽予警扣案，經警於104年6月2日通知吳鈞婷到案說明，始悉上情。


In [9]:
#m 數字 t 時間 c 連詞
#regex
#/○{2,}/
#/\d{10}/
#/[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}/ 車牌
import jieba.posseg as pseg

In [10]:
words =pseg.cut(tf_idf.get_content(1500))
for w in words:
    print(w.word,w.flag)
    break


謝鈞 nr


In [2]:
import numpy as np
import pandas as pd
import re
from datetime import datetime

#my_regex = "\（.*?\）|\(([^\)0-9零一二三四五六七八九十]?|[^\).]{2,})\)"

#tmpstr = '丙○○意圖為自己不法之所有，(甲)YY(假)XX(十一)(一)a(一一)b(一一一)c(年一)d(一年一)e(一一一年)A(9年)B(99年)C(999年)A(AAA)於民國105年10月8日晚間7時20分許，騎乘登記於不知情之王介平名下之車牌號碼000-000號普通重型機車，行經高雄市○○區○○○路000號前時，將口罩脫下遮掩車牌，自後靠近步行在人行道上之甲○○，趁甲○○不及防備之際，以左手徒手掠取甲○○右手持有之背包1只（內有現金新臺幣【下同】9500元、索尼牌ZR綠色及Xperformance金色行動電話共2具【IMEI：000000000000000、000000000000000號】、長夾1個、信用卡及金融卡共5張、國民身分證2張、國民健康保險卡1張、普通重型機車駕照1張、識別證2張等物）得手後逃逸。嗣因丙○○將背包內之現金取走後，將背包棄置在高雄市鳳山區武營路某處，再經民眾在高雄市鳳山區衛武營附近拾獲上開背包（內有長夾1個、國民身分證2張、國民健康保險卡1張、信用卡3張、金融卡1張等物），遂送交警察機關發還甲○○領回。(二)於同年10月10日晚間11時5分許，騎乘向不知情之劉伊恩借用之車牌號碼000-0000號普通重型機車，行經高雄市○○區○○○路000號前時，使用口罩遮掩車牌，自後靠近步行在人行道上之少年乙○○（89年生，基於兒童及少年福利與權益保障法第69條第2項規定不得揭露足以識別少年身分之資訊，年籍詳卷），趁乙○○不及防備之際，以左手徒手掠取乙○○右手持有之背包1只（內有皮夾1只、現金3萬2000元、提款卡1張、國民身分證1張、國民健康保險卡1張等物）得手後逃逸，並將現金取出，其餘物品則丟棄。嗣為警循線於105年10月16日凌晨2時許，在高雄市○○區○○路000巷00號前查獲，並扣得現金2600元。'
#print((tmpstr))
#print(len(tmpstr))
#print((re.sub(my_regex, "", tmpstr)))
#print(len(re.sub(my_regex, "", tmpstr)))

In [3]:
dt_idf = pd.read_csv(filedir + "\\Result_TFIDF_" + filenum +".csv", encoding='utf8')

In [4]:
dt_idf.head()

,詞,詞性,P
0,冠帽,n,3.299725
1,台宜,ns,3.299725
2,樣之運,n,3.299725
3,挪威,ns,3.299725
4,王維燦,nr,3.299725


In [5]:
dt_idf[dt_idf["P"] > p_upper].count()

詞     7262
詞性    7262
P     7262
dtype: int64

In [6]:
dt_idf.count()

詞     14081
詞性    14081
P     14081
dtype: int64

In [7]:
dt_idf['P'] = dt_idf['P'].astype('float')

In [8]:
dt_idf = dt_idf[dt_idf["P"] > p_upper].drop(columns=['詞性']).set_index('詞').T.to_dict('list')

In [9]:
df=pd.read_csv(filedir + "\\Result_" + filenum +".csv", index_col=0)
#df=pd.read_csv('.//data//Result_R5.csv', index_col=0)
df.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
3086,臺灣高雄地方法院,2016-05-09T00:00:00+08:00,"105,智簡,32",刑事,違反商標法,判決,吳鈞婷犯商標法第九十七條之非法陳列侵害商標權之商品罪，處有期徒刑貳月，如易科罰金，以新臺幣壹...,臺灣高雄地方法院刑事簡易判決105年度智簡字第32號聲請人臺灣高雄地方法院檢察署檢察官被告吳...,吳鈞婷明知註冊審定號第00000000號商標圖樣係商標權人美商新時代冠帽公司（下稱新時代公司...
5280,臺灣臺北地方法院,2018-02-01T00:00:00+08:00,"106,訴,519",刑事,毒品危害防制條例,判決,林義芳犯藥事法第八十三條第一項之轉讓偽藥罪，處有期徒刑肆月。\n扣案如附表所示之物沒收。,臺灣臺北地方法院刑事判決106年度訴字第519號公訴人臺灣臺北地方法院檢察署檢察官被告林義芳...,林義芳明知愷他命係毒品危害防制條例所列管之第三級毒品，且經改制前之行政院衛生署（現改制為衛生...


In [10]:
df['reason'] = df['reason'].str.replace('等','')
df['reason'] = df['reason'].str.replace('罪','')
df['reason'] = df['reason'].str.replace("條例","")
df['reason'] = df['reason'].str.replace("防制","")
df['reason'] = df['reason'].str.replace("管制","")
df['reason'] = df['reason'].str.replace("違反","")
df.loc[:,'reason'].value_counts()

公共危險      242
毀損        240
妨害名譽      236
竊盜        232
毒品危害      225
著作權法      224
妨害風化      223
妨害公務      218
妨害性自主     201
妨害自由      193
槍砲彈藥刀械    174
侵占        171
傷害        169
偽造文書      167
賭博        160
搶奪        156
過失傷害      155
殺人未遂      147
商標法       127
詐欺        124
強盜        112
殺人         30
毀棄損壞       26
恐嚇         19
恐嚇取財       15
過失致死        5
貪污          4
醫師法         2
藥事法         2
替代役實施       1
Name: reason, dtype: int64

In [11]:
my_regex = r"[\(\（]([^一二三四五六七八九零十1234567890]{1}[^\)\）]+|[一二三四五六七八九零十1234567890]{1}[^\)\）]{1,})[\)\）]"
df['Situation'] = df['Situation'].str.replace(my_regex, "")
#/○{2,}/
regex1 = r"/○{2,}"
df['Situation'] = df['Situation'].str.replace(regex1, "")

#/\d{10}/
regex2 = r"\d{10}"
df['Situation'] = df['Situation'].str.replace(regex2, "")

#/[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}/ 車牌
regex3 = r"[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}"
df['Situation'] = df['Situation'].str.replace(regex3, "")

In [12]:
run = 0
today = datetime.now()
print(run, ' => ', today)
for index, row in df.iterrows():
    try:
        words =pseg.cut(row.Situation)
        seq = ''
        for w, l in words:
            if (w in dt_idf):
                continue
            elif (l in ('m', 't', 'c')):
                continue
            else:
                seq += w
        df.loc[index, 'Situation'] = seq
        run += 1
        if (run % 1000 == 0):
            today = datetime.now()
            print(run, ' => ', today)
    except:
        continue

df.head(2)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.766 seconds.
Prefix dict has been built succesfully.


0  =>  2020-12-28 14:27:56.108386
1000  =>  2020-12-28 14:38:28.361062
2000  =>  2020-12-28 14:48:00.568607
3000  =>  2020-12-28 14:57:31.985428


,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
3086,臺灣高雄地方法院,2016-05-09T00:00:00+08:00,"105,智簡,32",刑事,商標法,判決,吳鈞婷犯商標法第九十七條之非法陳列侵害商標權之商品罪，處有期徒刑貳月，如易科罰金，以新臺幣壹...,臺灣高雄地方法院刑事簡易判決105年度智簡字第32號聲請人臺灣高雄地方法院檢察署檢察官被告吳...,吳鈞婷明知註冊審定號商標圖樣係商標權人美商新時代公司，向我國經濟部智慧財產局申請註冊，取得商...
5280,臺灣臺北地方法院,2018-02-01T00:00:00+08:00,"106,訴,519",刑事,毒品危害,判決,林義芳犯藥事法第八十三條第一項之轉讓偽藥罪，處有期徒刑肆月。\n扣案如附表所示之物沒收。,臺灣臺北地方法院刑事判決106年度訴字第519號公訴人臺灣臺北地方法院檢察署檢察官被告林義芳...,林義芳明知愷他命係毒品危害防制條例所列管之級毒品，經改制前之行政院衛生署公告列為管制藥品，不...


In [13]:
df.to_csv(filedir + "\\result_Sbert_" + filenum +"_TFIDF_" + p_upperT + "X.csv", index=0)